In [1]:

!pip uninstall transformers accelerate -y
!pip install transformers[torch] accelerate


Found existing installation: transformers 4.42.4
Uninstalling transformers-4.42.4:
  Successfully uninstalled transformers-4.42.4
Found existing installation: accelerate 0.32.1
Uninstalling accelerate-0.32.1:
  Successfully uninstalled accelerate-0.32.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using

In [2]:
import os
import re
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from shutil import copyfile
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoFeatureExtractor, AutoModelForImageClassification, TrainingArguments, Trainer
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
from transformers import ViTForImageClassification, Trainer, TrainingArguments
import cv2
from transformers import ViTForImageClassification, ViTConfig, TrainingArguments, Trainer
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import ViTForImageClassification, ViTConfig, TrainingArguments, Trainer
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1.데이터 불러오기

In [4]:
# 데이터셋 파일 경로
dataset_path = r'/content/drive/MyDrive/Colab Notebooks/관세청/updated_final_dataset.xlsx'

# Excel 파일 전체를 데이터프레임으로 불러오기
df1 = pd.read_excel(dataset_path)

# 데이터프레임 출력
print(df1)

                                    Name  \
0      Elite 스트레이트 진 | 7 For All Mankind   
1           로우라이즈 부츠컷 진  | Saint Laurent   
2                   Marlo 스트레이트 진 | Tove   
3       Nedar 하이라이즈 스트레이트 진 | Loro Piana   
4                       스트레이트 진 | Toteme   
...                                  ...   
17661                                NaN   
17662                                NaN   
17663                                NaN   
17664                                NaN   
17665                                NaN   

                                               Image URL  label  
0      https://img.mytheresa.com/652/738/100/jpeg/cat...   6204  
1      https://img.mytheresa.com/652/738/100/jpeg/cat...   6204  
2      https://img.mytheresa.com/652/738/100/jpeg/cat...   6204  
3      https://img.mytheresa.com/652/738/100/jpeg/cat...   6204  
4      https://img.mytheresa.com/652/738/100/jpeg/cat...   6204  
...                                                  ...    ...  
17661    

# 2.데이터 정리

## 1) 엑셀데이터 정리

In [5]:
df1.head()

,Name,Image URL,label
0,Elite 스트레이트 진 | 7 For All Mankind,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
1,로우라이즈 부츠컷 진 | Saint Laurent,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
2,Marlo 스트레이트 진 | Tove,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
3,Nedar 하이라이즈 스트레이트 진 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
4,스트레이트 진 | Toteme,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204


In [6]:
df1.tail()

,Name,Image URL,label
17661,NaN,NaN,6102
17662,NaN,NaN,6102
17663,NaN,NaN,6102
17664,NaN,NaN,6102
17665,NaN,NaN,6102


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17666 entries, 0 to 17665
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       11954 non-null  object
 1   Image URL  11954 non-null  object
 2   label      17666 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 414.2+ KB


행이 0부터 시작하여 이미지 파일명과 동일하게 행번호 변경

In [8]:
# 인덱스 변경
df1.index = range(2, 2 + len(df1))
df1

,Name,Image URL,label
2,Elite 스트레이트 진 | 7 For All Mankind,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
3,로우라이즈 부츠컷 진 | Saint Laurent,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
4,Marlo 스트레이트 진 | Tove,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
5,Nedar 하이라이즈 스트레이트 진 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
6,스트레이트 진 | Toteme,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
...,...,...,...
17663,NaN,NaN,6102
17664,NaN,NaN,6102
17665,NaN,NaN,6102
17666,NaN,NaN,6102


## 2) 이미지 데이터 정리

In [9]:
# 주어진 라벨을 새로운 숫자 라벨로 매핑할 딕셔너리 생성
label_mapping = {6101: 0, 6102: 1, 6103: 2, 6104: 3,
                 6105: 4, 6106: 5, 6109: 6, 6110: 7,
                 6201: 8, 6202: 9, 6203: 10, 6204: 11,
                 6205: 12, 6206:13 }

# 기존 라벨을 새로운 숫자 라벨로 변환하여 새로운 열 추가
df1['new_label'] = df1['label'].map(label_mapping)

# 변환된 데이터셋을 이용하여 데이터 분할
# 먼저 변환된 라벨을 가져오기
new_labels = df1['new_label']
df1

,Name,Image URL,label,new_label
2,Elite 스트레이트 진 | 7 For All Mankind,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204,11
3,로우라이즈 부츠컷 진 | Saint Laurent,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204,11
4,Marlo 스트레이트 진 | Tove,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204,11
5,Nedar 하이라이즈 스트레이트 진 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204,11
6,스트레이트 진 | Toteme,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204,11
...,...,...,...,...
17663,NaN,NaN,6102,1
17664,NaN,NaN,6102,1
17665,NaN,NaN,6102,1
17666,NaN,NaN,6102,1


In [10]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer

# 필요한 라이브러리 설치 (필요시)
# !pip install tensorflow tqdm

# 이미지 경로 설정
folder_path = '/content/drive/MyDrive/Colab Notebooks/관세청/final_image'

# 이미지 데이터 불러오기 및 전처리
image_size = (128, 128)  # 이미지 크기 조정
images = []
labels = []

# 사용할 확장자 목록
valid_extensions = ['.png', '.jpg', '.jpeg', '.PNG']


# 인덱스를 정렬하지 않고 순서대로 이미지 파일을 불러오기
for idx in tqdm(df1.index, desc="Loading images"):
    base_filename = f'{idx}'
    img_path_found = False

    for ext in valid_extensions:
        img_path = os.path.join(folder_path, base_filename + ext)
        if os.path.exists(img_path):
            img = tf.io.read_file(img_path)
            img = tf.image.decode_image(img, channels=3)
            img = tf.image.resize(img, image_size)
            images.append(img.numpy())  # TensorFlow 텐서를 NumPy 배열로 변환
            labels.append(df1.loc[idx, 'new_label'])  # df1의 'new_label' 컬럼 사용
            img_path_found = True
            break  # 이미지가 발견되면 루프를 중지

    if not img_path_found:
        print(f"Warning: Image file for index {idx} not found in any supported format.")

# 데이터를 NumPy 배열로 변환
images = np.stack(images)

Loading images: 100%|██████████| 17666/17666 [3:15:50<00:00,  1.50it/s]


In [11]:
# images 리스트의 개수 확인
num_images = len(images)
print(f'Total number of images: {num_images}')

Total number of images: 17666


#3. 데이터 분리

1)이미지 데이터 (x값)

2) 엑셀 데이터 (y값)

In [12]:
# 랜덤 시드 설정
np.random.seed(42)
torch.manual_seed(42)

# 데이터 분할 비율 설정
train_ratio = 0.7  # 학습 데이터 비율
valid_ratio = 0.2  # 검증 데이터 비율
test_ratio = 0.1  # 테스트 데이터 비율

# 데이터 수 계산
total_images = len(images)
num_train = int(total_images * train_ratio)
num_valid = int(total_images * valid_ratio)
num_test = total_images - num_train - num_valid

# 데이터를 나눠서 저장
x_train = images[:num_train]
x_valid = images[num_train:num_train + num_valid]
x_test = images[num_train + num_valid:]

# 라벨을 NumPy 배열로 변환
labels = np.array(labels)

# 데이터를 나눔
y_train = labels[:num_train]
y_valid = labels[num_train:num_train + num_valid]
y_test = labels[num_train + num_valid:]

# 각 데이터셋의 크기 출력
print(f"전체 이미지 수: {total_images}")
print(f"학습 데이터 수: {len(x_train)}")
print(f"검증 데이터 수: {len(x_valid)}")
print(f"테스트 데이터 수: {len(x_test)}")
print("---")
print(f"학습 데이터 수: {len(y_train)}")
print(f"검증 데이터 수: {len(y_valid)}")
print(f"테스트 데이터 수: {len(y_test)}")

전체 이미지 수: 17666
학습 데이터 수: 12366
검증 데이터 수: 3533
테스트 데이터 수: 1767
---
학습 데이터 수: 12366
검증 데이터 수: 3533
테스트 데이터 수: 1767


# 4.데이터셋 준비

In [13]:
# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.fromarray((self.images[idx] * 255).astype(np.uint8))  # 이미지 배열을 PIL 이미지로 변환
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return {
            'pixel_values': image,    # 이미지 데이터
            'labels': torch.tensor(label).long()  # 라벨 데이터를 텐서로 변환
        }

# 이미지 resize를 위한 transform 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # 이미지를 (224, 224) 크기로 resize
    transforms.ToTensor(),           # 이미지를 Tensor로 변환 (0 ~ 1 사이의 값으로 정규화)
])

# 데이터셋 객체 생성
train_dataset = CustomDataset(x_train, y_train, transform=transform)
valid_dataset = CustomDataset(x_valid, y_valid, transform=transform)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)

# 5.사전학습 모델 불러오기 및 전이학습

In [15]:
# 모델 설정
model_name = "google/vit-base-patch16-224-in21k"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=14)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',                 # 학습 결과를 저장할 디렉토리 경로
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    logging_dir='./logs',
)

# Trainer 객체 생성 및 학습 시작
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,      # 학습 데이터셋 설정
    eval_dataset=valid_dataset,       # 검증 데이터셋 설정
)

# 학습 실행
trainer.train()

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,1.411700
1000,0.934500
1500,0.823900
2000,0.651000
2500,0.645100
3000,0.622100
3500,0.506500
4000,0.481000
4500,0.490400
5000,0.383600


Step,Training Loss
500,1.411700
1000,0.934500
1500,0.823900
2000,0.651000
2500,0.645100
3000,0.622100
3500,0.506500
4000,0.481000
4500,0.490400
5000,0.383600


TrainOutput(global_step=46380, training_loss=0.14327116343624482, metrics={'train_runtime': 14200.8997, 'train_samples_per_second': 26.124, 'train_steps_per_second': 3.266, 'total_flos': 2.875107021565944e+19, 'train_loss': 0.14327116343624482, 'epoch': 30.0})

#6. 모델 평가

In [17]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# MyDataset 클래스 정의 (PCA 적용을 제거)
class MyDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = np.array(images)  # 이미지를 넘파이 배열로 변환
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.fromarray((self.images[idx] * 255).astype(np.uint8))  # 이미지 배열을 PIL 이미지로 변환
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return {
            'pixel_values': image,  # 이미지 데이터
            'labels': torch.tensor(label, dtype=torch.long)  # 라벨 데이터를 텐서로 변환
        }

# 이미지 resize를 위한 transform 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지를 (224, 224) 크기로 resize
    transforms.ToTensor(),          # 이미지를 Tensor로 변환 (0 ~ 1 사이의 값으로 정규화)
])

# 데이터셋 객체 생성
test_dataset = MyDataset(x_test, y_test, transform=transform)

# 예측 수행
predictions = trainer.predict(test_dataset)

# 예측 결과 추출
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# 정확도와 F1 점수 계산
accuracy = accuracy_score(labels, preds)
f1 = f1_score(labels, preds, average='weighted')
precision = precision_score(labels, preds, average='weighted')
recall = recall_score(labels, preds, average='weighted')

print(f"Test Accuracy: {accuracy}")
print(f"Test F1 Score: {f1}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")

Test Accuracy: 0.2569326542161856
Test F1 Score: 0.31032426373406236
Test Precision: 0.4001600709443831
Test Recall: 0.2569326542161856


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


vit_의류 모델의 성능은 0.2569(정확도), 0.4001(정밀도), 0.2569(재현율), 0.3103(F1 Score)로 나타났다.